 ### In Spark SQL, there are two types of tables: 
- Managed Tables (Internal)

- Unmanaged Tables (External)

##### Consider we have this file

In [ ]:
from pyspark.sql.types import *

order_csv_file_dir = "Files/csv/orders/2019.csv"

orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
    ])

csvDF = spark.read.format("csv").schema(orderSchema).load(order_csv_file_dir)
display(csvDF.limit(3))

##### Managed Tables (Internal) - Example
- **Managed Tables (Internal):** When you create a managed table, Spark manages both the data and the metadata. In other words, it not only stores the metadata information about the table in the Hive Metastore, but it also stores the actual data in the path of the Hive warehouse. If you drop a managed table, both the metadata and the actual data are deleted.

In [ ]:
# No need to mention the file path directoru name
DeltaTableName = "Sales_SMT"

# Save as a Delta files
(
    csvDF.write.format("parquet")
         .mode("overwrite")
         .option("overwriteSchema", "true")
         .saveAsTable(DeltaTableName)
)

##### Verify if its a managed table

In [ ]:
spark.catalog.listTables()

##### Get schema definition and delta file location

In [ ]:
%%sql
DESCRIBE EXTENDED Sales_SMT

##### Display the file(s) name

In [ ]:
files = mssparkutils.fs.ls("abfss://e4b530ef-cb86-40d5-8d6b-fe9152659677@onelake.dfs.fabric.microsoft.com/2c99179f-673c-441d-89b6-021c0f9ed71f/Tables/sales_smt") 

for file in files:
    print(file.name)
    #print(file.name, file.isDir, file.isFile, file.path, file.size)

##### Drop Delta Managed Table

In [ ]:
%%sql

DROP TABLE Sales_SMT

##### Will fail as the file is no more existed

In [ ]:
files = mssparkutils.fs.ls("abfss://e4b530ef-cb86-40d5-8d6b-fe9152659677@onelake.dfs.fabric.microsoft.com/2c99179f-673c-441d-89b6-021c0f9ed71f/Tables/sales_smt") 

for file in files:
    print(file.name)
    #print(file.name, file.isDir, file.isFile, file.path, file.size)

##### Unmanaged Tables (External) - Example
- **Unmanaged Tables (External):** When you create an unmanaged table, Spark only manages the metadata (i.e., the information about the table stored in the Hive Metastore). The data itself is stored in an external location, usually a path in HDFS or another storage system. If you drop an unmanaged table, only the metadata is deleted, while the data itself remains intact.

In [ ]:
%%pyspark
# Example: Managed Table (External)


deltaTablePath = "Files/tmpfiles/Sales"

DeltaTableName = "Sales_SUMT"
# Save as a Parquet files
(
    csvDF.write.format("parquet")
         .mode("overwrite")
         .option("overwriteSchema", "true")
         .option("path", deltaTablePath)
         .saveAsTable(DeltaTableName)
)

##### Verify if its a unmanaged table

In [ ]:
spark.catalog.listTables()

##### Drop Delta Managed Table

In [ ]:
%%sql

DROP TABLE Sales_SUMT

In [ ]:
spark.catalog.listTables()

In [ ]:
# Delete Delta Table (folder)
mssparkutils.fs.rm(deltaTablePath, recurse=True)

In [ ]:
%%sql

DROP TABLE purchaseorderdetail_dp203

In [ ]:
# Delete Delta Table (folder)
delta_table_path = "Files/deltafiles/BicycleSales"
mssparkutils.fs.rm(delta_table_path, recurse=True)